In [ ]:
import sys

sys.path.append('..')

In [ ]:
from dataclasses import dataclass, field
from pathlib import Path
from pprint import pp

from herbarium.pylib import db
from herbarium.pylib.multi_efficient_net import NETS
from herbarium.pylib.run_model import test

In [ ]:
DATA = Path('..') / 'data'
MODELS = DATA / 'models'

In [ ]:
# TRAITS = """ flowering fruiting """.split()
TRAITS = ['flowering']
# TRAITS = ['fruiting']

In [ ]:
@dataclass
class Args:
    net: str = ''
    test_run: str = ''
    split_run: str = ''
    load_weights: Path = MODELS
    trait: list[str] = field(default_factory=list)
    database: Path = DATA / 'angiosperms.sqlite'
    limit: int = 0
    workers: int = 8
    batch_size: int = 8

In [ ]:
for trait in TRAITS:
    paths = (MODELS / trait).glob('*.pth')
    for path in sorted(paths):
        p = path.stem.split('_')

        args = Args(
            net=p[0],
            test_run=path.stem,
            split_run=f'{p[1]}_2_orders',
            load_weights=path,
            trait=[trait],
        )

        orders = db.select_orders(args.database, args.split_run)

        net = NETS[args.net](len(orders), args.load_weights, freeze='all')

        print(path.stem, flush=True)

        test(args, net, orders)

        print()